In [1]:
from langchain.tools import DuckDuckGoSearchResults
from bs4 import BeautifulSoup
import requests
from extracter import LinkExtractor
from links_generatior import get_links

In [46]:
queery= "Gather information on Canoo's financial performance, including its revenue, profit margins, return oninvestment, and expense structure. " 

In [47]:
getlink= get_links(queery)
links=getlink.links_generator()

In [48]:
links

['https://investors.canoo.com/financial-information/financial-results',
 'https://investors.canoo.com/news-presentations/press-releases/detail/100/canoo-inc-announces-fourth-quarter-and-fiscal-year-2022',
 'https://investors.canoo.com/news-presentations/press-releases/detail/117/canoo-inc-announces-third-quarter-2023-results',
 'https://www.prnewswire.com/news-releases/canoo-inc-announces-third-quarter-2023-results-301988195.html']

In [49]:
#langchain async html loader 
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer

In [50]:
loader = AsyncHtmlLoader(links)
docs = loader.load()

Fetching pages:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 4/4 [00:03<00:00,  1.07it/s]


In [51]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [ ]:
docs_transformed

In [53]:
results=docs_transformed[1].page_content

In [54]:
total_text=""
total_text+=results

In [55]:
import csv

def save_to_csv(final_text, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([final_text])

# Example usage:

filename = "fourth_quest.csv"
save_to_csv(results, filename)

In [58]:
file_path = "output.txt"
with open(file_path, "w") as file:
    file.write(total_text)

Vector Store Implementation for Retrival and question answering

In [60]:

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain import OpenAI

In [61]:
import os
os.environ["OPENAI_API_KEY"]=""

In [62]:
path="F:\langchain\webscrapper\output.txt"
with open(path, "r") as file:
    loaded_text = file.read()

In [64]:
text_split= CharacterTextSplitter(
    separator='\n',
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)
splitted_text= text_split.split_text(loaded_text)

In [ ]:
len(splitted_text)

In [67]:
embeddings= OpenAIEmbeddings()

In [68]:
vectorstore= FAISS.from_texts(splitted_text, embeddings)

In [69]:
vectorstore.save_local("vector_embedding")

In [70]:
new_db= FAISS.load_local("F:\langchain\webscrapper\\vector_embedding",embeddings)

In [ ]:
new_db

In [ ]:
new_db.similarity_search("Tell me about Canno")

In [79]:
from langchain.chains import RetrievalQA


reteiver= new_db.as_retriever(search_type="similarity",search_kwargs={"k":4})
rqa=RetrievalQA.from_chain_type(llm=OpenAI(),
                                chain_type="stuff",
                                retriever=reteiver,
                                return_source_documents=True)

In [ ]:
rqa("What is finalcial conditions of Canoo")["result"]